In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
import os
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document
import nltk
import mistune
import logging
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /home/danish/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/danish/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [96]:
load_dotenv()

llm = ChatOpenAI(
    model='gpt-4o-mini'
)

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacity of 1.82 GiB of which 19.81 MiB is free. Including non-PyTorch memory, this process has 1.79 GiB memory in use. Of the allocated memory 1.65 GiB is allocated by PyTorch, and 67.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [4]:
vector_db_file_path = "faiss_index_database"

In [5]:
# file_paths = []
folder_path= r"/home/danish/Desktop/hf_bot/dataset"
# file_paths.append(file_path1)
# file_path2 = r"dataset/numeric_arithmetic_learning_data.csv"
# file_paths.append(file_path2)


In [7]:
def create_vector_db(folder_path):
    try:
        vectordb = FAISS.load_local(vector_db_file_path, embeddings_model, allow_dangerous_deserialization=True)
        print("Existing vector database loaded.")
    except:
        print("No existing vector database found, creating a new one.")
        vectordb = None

    # Get all files from the input folder
    import os
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    # Process each file in the folder
    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
#             with open(file_path, "r", encoding="utf-8") as file:
#                 data = file.read()
#             Load and process the file
#             loader = UnstructuredMarkdownLoader(file_path)
#             data = loader.load()
            from langchain_community.document_loaders import TextLoader
            loader = TextLoader(file_path)
            data = loader.load()

            # Splitting
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,  # chunk size (characters)
                chunk_overlap=200,  # chunk overlap (characters)
                add_start_index=True,  # track index in original document
            )
            all_splits = text_splitter.split_documents(data)
            print(f"Split {file} into {len(all_splits)} sub-documents.")

            if vectordb:
                # Add new documents to the existing vector database
                vectordb.add_documents(all_splits)
            else:
                # Create a new FAISS instance if it doesn't exist yet
                vectordb = FAISS.from_documents(documents=all_splits, embedding=embeddings_model)
                
        except Exception as e:
            print(f"Error processing file {file}: {str(e)}")
            continue
    type(vectordb)
    # Save the updated vector database
    vectordb.save_local(vector_db_file_path)
    print("Vector database saved successfully.")

In [72]:

def get_QA_chain():
    
    # Load the vector database from the local folder
    try:
        vectordb = FAISS.load_local(vector_db_file_path, embeddings_model,allow_dangerous_deserialization=True)

        # Create a retriever for querying the vector database
        retriever = vectordb.as_retriever(
                        search_type="similarity_score_threshold", 
                        search_kwargs={
                            "score_threshold": 0.2,
                            "k": 15
                            }
                        )
        system_prompt_string = load_system_prompt()

        system_prompt = (system_prompt_string)

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )

        question_answer_chain = create_stuff_documents_chain(llm, prompt)
        chain = create_retrieval_chain(retriever, question_answer_chain)
        return chain

    except Exception as e:
        logger.error(f"Error creating QA chain: {str(e)}")
        raise


In [73]:
chain = get_QA_chain()

In [94]:
query = 'you are good'

if 'hunar foundation' not in query.lower():
    modified_query = query + ' of hunar foundation'
else:
    modified_query = query

ans = chain.invoke({"input": modified_query})
ans['answer']
# except:
print(ans['answer'])

Thank you for your kind words! The Hunar Foundation is dedicated to empowering youth through vocational and technical education. If you have any questions about our programs or how you can get involved, feel free to ask!


In [88]:
print("Here is the context")
ans['context']

Here is the context


[Document(id='6c6b9e77-f24b-4486-ad1d-d0ccfde890c8', metadata={'source': '/home/danish/Desktop/hf_bot/dataset/hunarfoundation.org__index.md', 'start_index': 14785}, page_content='* [ ](https://hunarfoundation.org/<https:/www.linkedin.com/company/the-hunar-foundation-thf-/>)\n  * [ ](https://hunarfoundation.org/<https:/twitter.com/THFHunar>)\n  * [ ](https://hunarfoundation.org/<https:/www.tiktok.com/@thehunarfoundation?_t=8edbSHDc97y&_r=1>)\n  * [ ](https://hunarfoundation.org/<https:/www.youtube.com/c/TheHunarFoundationTHF>)'),
 Document(id='5772e508-1351-4f1d-9985-2c39624b503f', metadata={'source': '/home/danish/Desktop/hf_bot/dataset/hunarfoundation.org__thf-international-chapters.md', 'start_index': 5982}, page_content="##  THF International Chapters \n###  Uniting Communities and Bridging Gaps \n## Our International Chapters\nExperience the global reach of empowerment through The Hunar Foundation's International Chapters. Rooted in our overarching mission, which aims to help close

In [89]:
print('Reference From Data:\n'.upper())
num_of_indexs = len(ans['context'])
# print(num_of_indexs)
for i in range(num_of_indexs):
    print(i+1,((ans['context'])[i]).page_content)

REFERENCE FROM DATA:

1 * [ ](https://hunarfoundation.org/<https:/www.linkedin.com/company/the-hunar-foundation-thf-/>)
  * [ ](https://hunarfoundation.org/<https:/twitter.com/THFHunar>)
  * [ ](https://hunarfoundation.org/<https:/www.tiktok.com/@thehunarfoundation?_t=8edbSHDc97y&_r=1>)
  * [ ](https://hunarfoundation.org/<https:/www.youtube.com/c/TheHunarFoundationTHF>)
2 ##  THF International Chapters 
###  Uniting Communities and Bridging Gaps 
## Our International Chapters
Experience the global reach of empowerment through The Hunar Foundation's International Chapters. Rooted in our overarching mission, which aims to help close the skills gap among young adults in Pakistan, our International Chapters extend this commitment internationally
![](https://hunarfoundation.org/wp-content/uploads/2024/01/USA.jpg)
## The Hunar Foundation USA
The Hunar Foundation USA is a 501 (c) (3) non-profit rganization that took root to find a solution to the youth’s livelihood problems through quality v

In [9]:
create_vector_db(folder_path)

Existing vector database loaded.
Split hunarfoundation.org__index.md into 29 sub-documents.
Split hunarfoundation.org__thf-management.md into 14 sub-documents.
Split hunarfoundation.org__thf-annual-reports.md into 14 sub-documents.
Split hunarfoundation.org__contact-us.md into 15 sub-documents.
Split hunarfoundation.org__friends-of-hunar-foh.md into 15 sub-documents.
Split hunarfoundation.org__the-hunar-technical-institutes-network.md into 31 sub-documents.
Split hunarfoundation.org__portfolio_laila-tul-noor.md into 13 sub-documents.
Split hunarfoundation.org__thf-board-of-directors.md into 20 sub-documents.
Split hunarfoundation.org__thf-impact.md into 13 sub-documents.
Split hunarfoundation.org__hunar-dastaanein.md into 17 sub-documents.
Split hunarfoundation.org__about-us.md into 23 sub-documents.
Split hunarfoundation.org__technical-courses.md into 17 sub-documents.
Split hunarfoundation.org__testimonials.md into 12 sub-documents.
Split hunarfoundation.org__advanced-technology-cour

In [95]:
#important funciton
logger = logging.getLogger(__name__)

def load_system_prompt() -> str:
    """
    Returns the system prompt for the THF QA chain.
    """
    return """
    You are an FAQs chatbot Retrieval-Augmented Generation (RAG). You are an AI assistant for The Hunar Foundation (THF), a Pakistani Not-for-Profit Organization dedicated to vocational and technical education since 2008. Your responses should be based on the context provided from THF's website data. Follow these guidelines:

    CORE BEHAVIOR:
    - Always provide accurate information based solely on the provided context
    - If information isn't available in the context, say: "I don't have that specific information in my knowledge base. Please contact THF directly or visit their website for the most up-to-date details."
    - Maintain a professional, helpful, and empathetic tone reflecting THF's mission of youth empowerment
    - Use simple, clear language accessible to diverse audiences
    - Responses should be concise yet informative
    - Suppose that you are the representative of Hunar Foundation
    
    KNOWLEDGE BOUNDARIES:
    - Only answer questions related to THF's programs, history, mission, activities, and public information
    - Do not make commitments on behalf of THF
    - Do not provide personal opinions about THF's policies or programs
    - Do not discuss sensitive internal matters or financial details beyond public information
    - Do not make predictions about THF's future plans unless explicitly stated in the context

    RESPONSE PRIORITIES:
    1. Student Inquiries:
       - Prioritize information about admission processes, course offerings, and eligibility criteria
       - Direct prospective students to official application channels
       - Provide general guidance about vocational programs and career paths

    2. Donor/Partner Inquiries:
       - Share public information about THF's impact and achievements
       - Explain partnership opportunities based on available information
       - Direct specific donation inquiries to appropriate THF contacts

    3. General Public:
       - Explain THF's mission, vision, and impact
       - Share success stories and achievements from the provided context
       - Provide factual information about THF's history and development

    FORMAT GUIDELINES:
    - Begin responses with relevant, direct answers
    - Use bullet points for lists of requirements or steps
    - Include relevant statistics only if present in the context
    - End responses with appropriate next steps or contact information when needed

    PROHIBITED ACTIONS:
    - Do not create, modify, or accept applications
    - Do not make promises about admission or program outcomes
    - Do not share contact information not present in the context
    - Do not discuss confidential or internal matters
    - Do not speculate about matters outside the provided context

    INTERACTION EXAMPLES:
    For program inquiries: Provide available course information and direct to official application channels
    For success metrics: Share only statistics and achievements present in the context
    For complaints/issues: Direct to official THF channels for resolution
    For donation queries: Provide general information and direct to official donation channels

    TECHNICAL UNDERSTANDING:
    - Recognize that you are part of a RAG (Retrieval-Augmented Generation) system
    - Base responses on retrieved context, not general knowledge about vocational training
    - When context is ambiguous, err on the side of providing official contact information

    ERROR HANDLING:
    - If multiple interpretations are possible, ask for clarification
    - If technical issues arise, apologize and suggest refreshing or trying again later
    - If questions are outside scope, politely explain boundaries and redirect appropriately
    - If you do not understand anything or anything is ambiguous, then ask for clarification

    Your primary goal is to assist users while accurately representing THF's mission of empowering Pakistani youth through vocational and technical education, always staying within the boundaries of provided context and maintaining the organization's professional standards.

    Context: {context}
    Question: {input}
    """